In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
#import warnings
#warnings.filterwarnings('ignore')

#import numpy as np
#import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
#import operator
#import fastparquet as fp 
#import google.cloud.storage as storage
#import scipy.stats as stats

In [2]:
modelo_games = pq.read_table("../0-DATA/modelo_games.parquet").to_pandas()
modelo_games.head(5)

,item_id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,643980,Ironbound,Free to Play
2,670290,Real Pool 3D - Poolians,Casual
3,767400,弹炸人2222,Action
4,772540,Battle Royale Trainer,Action


In [3]:
#Creamos los dummies
modelo_item = pd.get_dummies(modelo_games, columns=["genres"], prefix="")

modelo_item = modelo_item.groupby(["item_id","app_name"]).sum().reset_index()

modelo_item.head(4)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
similitudes = cosine_similarity(modelo_item.iloc[:,3:])


In [5]:
similitudes.shape


(27461, 27461)

In [6]:
print(similitudes)


[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


Para la elección de la similitud del conseno se puede hacer por varias vias:
    * Pandas: Por las caracteristicas de la formula solo es para 2 o 4 columnas a lo mucho 4 ya que la formula se haria engorrosa 
    * Numpy: Por las caracteristicas de la formula solo es para 2 o 4 columnas a lo mucho 4 ya que la formula se haria engorrosa 

In [7]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtramos el juego e igualarlo a  su ID
    juego_seleccionado = modelo_item[modelo_item['item_id'] == id]
    # devolvemos error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calculamos la matriz de similitud coseno
    #similitudes = cosine_similarity(modelo_item.iloc[:,3:])
    
    # Calculamos la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_item[modelo_item['item_id'] == id].index[0]]
    
    # Calculamos los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtenemos los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados

In [8]:
recomendacion_juego(2028055) #411350, 2028055, 3260, 100400, 218760, 3310, 229752, 290810, 388280, 754300, 
                            #766101, 766521, 387800, 462640, 1620, 244830, 429670, 485150


12683                Tempest: Pirate Action RPG
12726                  GGXrd System Voice - MAY
12725             GGXrd System Voice - KY KISKE
12724           GGXrd System Voice - SOL BADGUY
12723    GGXrd Character Unlock - LEO WHITEFANG
Name: app_name, dtype: object

Dividimos el dataframe por la mitad a fin de lograr montar el modelo en render



In [9]:
# Revisamos la cantidad de filas
cant_filas= len(modelo_item)

# Calculamos la mitad
mitad_filas= cant_filas // 10
 
# Seleccionamos la mitad superior
modelo_railway= modelo_item.iloc[:mitad_filas]

In [10]:
modelo_railway.shape


(2746, 24)

In [11]:
similitudes_railway = cosine_similarity(modelo_railway.iloc[:,3:])

In [12]:
tabla = pa.Table.from_pandas(modelo_railway) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"../0-DATA/modelo_railway.parquet") # guardamos la tabla en formato parquet

In [13]:
modelo_railway.to_csv("../0-DATA/modelo_railway.csv", index=False, encoding="utf-8") 

In [14]:
def encontrar_juegos_similares(id):
    # Encontramos el índice del juego ingresado por ID
    juego_indice = modelo_railway.index[modelo_railway['item_id'] == id].tolist()[0]
    
    # Extraemos las características del juego ingresado
    juego_caracteristicas = modelo_railway.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Calculamos la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_railway = cosine_similarity(modelo_railway.iloc[:, 3:], juego_caracteristicas)
    
    # Obtenemos los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_railway.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # Obtenemos los juegos más similares en función de los índices
    juegos_similares = modelo_railway.iloc[indices_juegos_similares]['app_name']
    
    return juegos_similares

In [15]:
encontrar_juegos_similares(3310)#3260, 3310, 1620, 10


2059    Rocksmith - My Chemical Romance - Planetary (GO!)
2060          Rocksmith - My Chemical Romance 3-Song Pack
2061                Rocksmith - Queen - Bohemian Rhapsody
2062                 Rocksmith - Queen - Stone Cold Crazy
Name: app_name, dtype: object

In [16]:
def recomendacion_juego(id: int):
    
    # Verificamos si el juego con game_id existe en df_games
    game = modelo_railway[modelo_railway['item_id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtenemos el índice del juego dado
    idx = game.index[0]

    # Tomamos una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Definimos el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_railway.sample(n=sample_size, random_state=42)  # Ajustamos la semilla aleatoria según sea necesario

    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_railway.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [17]:
recomendacion_juego(3310)


{'similar_games': ['Hidden Expedition: Titanic',
  'Dwarfs - F2P Difficulty Pack',
  'Rocksmith - Guitarcade - Time Saver Pack',
  'Virtual Villagers: A New Home',
  'The Otherside: Realm of Eons']}